## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2021-09-12 08:50:44--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2021-09-12 08:50:44--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd5783621b3119d3136014f050f.dl.dropboxusercontent.com/cd/0/get/BWDCxvLEmdSwIEzIUKWfgvVOej7KIvb7nW0F_NNJm_Qt8D2HULZmS3R92eAR0JFXI_6S6gItozgR1rbWgqhyrjePWsF9btW7ikOpRrIeXm4yjWuOYzl6ukPVs5wdSIk1fOTcdzuPa3FdGrIJl_bMAj7Q/file?dl=1# [following]
--2021-09-12 08:50:44--  https://ucd5783621b3119d3136014f050f.dl.dropboxusercontent.com/cd/0/get/BWDCxvLEmdSwIEzIUKWfgvVOej7KIvb7nW0F_NNJm_Qt8D2HULZmS3R92eAR0JFXI_6S6gItozgR1rbWgqhyrjePWsF9btW7i

In [2]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(t.lower()) for t in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-0.41975823, -0.43590358, -1.5973825 ,  0.67233145, -1.4326688 ,
        0.93201196,  2.5122323 ,  1.026866  ,  1.1710931 ,  0.22237958,
        2.0413446 , -1.2438407 ,  1.4944769 , -0.89928937, -1.8776034 ,
       -2.053043  ,  2.4185135 , -4.431299  , -4.536832  , -0.21418947,
       -1.1013223 ,  1.8827406 ,  2.3999865 , -3.9931157 ,  1.8662909 ,
       -1.3104059 , -3.9985828 ,  0.43505532,  1.9268502 , -1.5682174 ,
        0.25125146,  1.7606997 ], dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('friendship')

[('life', 0.8038187623023987),
 ('feelings', 0.7983453869819641),
 ('dreams', 0.788669228553772),
 ('feeling', 0.7751195430755615),
 ('relationship', 0.7751133441925049),
 ('soul', 0.7674163579940796),
 ('relationships', 0.7615863680839539),
 ('siblings', 0.7505725026130676),
 ('friends', 0.7496085166931152),
 ('eyes', 0.7440325021743774)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
model.most_similar(positive=["man", "car"], negative=["money"])

[('bus', 0.6903536319732666),
 ('driver', 0.626215934753418),
 ('front', 0.6073166728019714),
 ('truck', 0.6050291061401367),
 ('guy', 0.5917913913726807),
 ('dead', 0.5762036442756653),
 ('bike', 0.571462094783783),
 ('dad', 0.5701184272766113),
 ('cool', 0.5680974721908569),
 ('was', 0.5668057799339294)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [13]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [14]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [15]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)
word_vectors_pca -= np.mean(word_vectors_pca)
word_vectors_pca /= word_vectors_pca.std(0)

In [16]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [18]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [19]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
word_tsne -= np.mean(word_tsne)
word_tsne /= word_tsne.std(0)

In [20]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1112', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [21]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    phrase = phrase.lower()
    phrase_tokens = tokenizer.tokenize(phrase)

    words = []

    for w in phrase_tokens:
      if w in model.vocab:
        words.append(w)

    if words:
      return np.mean(np.array([model.get_vector(word) for word in words]),axis=0)
    return vector
        
    

In [22]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [23]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [24]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [25]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)
phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [26]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1232', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [27]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [28]:
from scipy.spatial import distance

def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    qvector = get_phrase_embedding(query)

    res = 1 - (data_vectors @ qvector) / np.linalg.norm(data_vectors, axis=1) / np.linalg.norm(qvector)
    idx = np.argsort(res)
    res = []
    for i in range(k):
      res.append(data[idx[i]])

    return res

In [29]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [30]:
find_nearest(query="How does Trump?", k=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [31]:
find_nearest(query="Why don't i ask a question myself?", k=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.

See what other embeddings are there in the model zoo: gensim.downloader.info():

In [32]:
 from gensim import downloader
 downloader.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

Optimize find_nearest with locality-sensitive hashing: use nearpy or sklearn.neighbors.

In [36]:
from sklearn.neighbors import KDTree

tree = KDTree(data_vectors) 
def find_nearest_locally_sensitive_hashing(query, tree=tree, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity is measured using KDTree algorithm
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    qvector = get_phrase_embedding(query)
    distances, indices = tree.query(qvector.reshape(1, -1), k=k)

    res = []
    for i in indices[0]:
      res.append(data[i])

    return res

In [37]:
data_vectors.shape
get_phrase_embedding("How do i enter the matrix?").reshape(1, -1).shape

(1, 100)

In [35]:
results = find_nearest_locally_sensitive_hashing(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
print(results[0:5])

How do I estimate the competition?
How do I use the pad in MMA?
What format do I use when writing a novel on Microsoft Word? What font do I use, how big are the letters? When do I indent? How do I set it all up?
How do you use the components in Final Fantasy XIII?
What should I do to enter hollywood?
How do I find the 5th dimension?
How do I solve this limit as n approaches infinity? What trick do I need to use in this?
How do I use openalpr in C++? How do I change the pattern according to the Indian licence plate? How do I compile it in Eclipse IDE?
What do I do to enter the line of event management?
How do I attach an HDD to the Xbox 360? And how do I copy games from the PC to the Xbox?

['How do I estimate the competition?\n', 'How do I use the pad in MMA?\n', 'What format do I use when writing a novel on Microsoft Word? What font do I use, how big are the letters? When do I indent? How do I set it all up?\n', 'How do you use the components in Final Fantasy XIII?\n', 'What should I 